Created by John Banovetz
Last Edited: 10/12/2023

This notebook shows how the 'raw' DC2 images were modified and then resaved as 'raw_modified' images for PREOPS-3761

In [1]:
from lsst.daf.butler import Butler

In [2]:
butler=Butler('/repo/dc2')
registry=butler.registry

In [3]:
# This was the latest reprocessing of DC2 data
dc2_collection=[
    '2.2i/runs/test-med-1/w_2023_34/DM-40459'
]

In [5]:
# Dectecor 88 was chosen as it is an e2V detector where the persistence is present
det=88
kwargs={
"datasetType": "raw",
"collections": dc2_collection,
"findFirst": True,
'detector': det,
'instrument': 'LSSTCam-imSim'
}

datasetRefs_dc2 = list(registry.queryDatasets(**kwargs))

In [ ]:
# Set run to where ever you want the output, this is the output for the process used in PREOPS-3761
butler_output=Butler('/repo/dc2',run='u/banovetz/dc2/w_2023_34/raw_persistence_single_value',writeable=True)

In [ ]:
#This sets up the new dataset type, might not have to be executed
original_raw_dst=butler_output.registry.getDatasetType("raw")
new_dst=DatasetType('raw_modified',original_raw_dst.dimensions,original_raw_dst.storageClass)

In [6]:
def tau_function(t):
#   Simple model for the persistence
    tau=37
    a=6
    return(a*np.exp(-t/tau))

def saturated_footprints(exp):
    # Detection method for finding the saturated pixels
    # Adapation of the lsst.summit.utils.detectObjectsinExp code
    threshold=140000
    grow=1
    nPixMin=5
    threshold = afwDetect.Threshold(threshold, afwDetect.Threshold.VALUE)
    footPrintSet = afwDetect.FootprintSet(exp.getMaskedImage(), threshold, "DETECTED", nPixMin)
    if grow > 0:
        isotropic = True
        footPrintSet = afwDetect.FootprintSet(footPrintSet, grow, isotropic)
    return(footPrintSet)

In [ ]:
# This is the for loop that runs the changes. Currently does only 100 of the 345 images and saves the 'raw_modified' images to the run from above.
times=[]
fptset_all=[]
for i in range(0,101):
    if i==0:
        check=butler.get(datasetRefs_2[i])
        times.append(0)
        fptset=saturated_footprints(check)
        fptset_all.append(fptset)
    else:
        og=butler.get(datasetRefs_2[i])
        check=og.clone()
        dark=np.zeros_like(check.image.array)
        for ind,(time,fptset) in enumerate(zip(times,fptset_all)):
            persist=tau_function(time)
            for fpt in fptset.getFootprints():
                x1=fpt.getBBox().beginX
                x2=fpt.getBBox().endX
                y1=fpt.getBBox().beginY
                y2=fpt.getBBox().endY
                sat_arr=fpt.spans.asArray()
                float_arr=np.zeros_like(sat_arr,dtype='float')
                for y,row in enumerate(sat_arr):
                    for x,val in enumerate(row):
                        if val==True:
                            float_arr[y][x]=persist
                        else:
                            float_arr[y][x]=0 
                yc=int((y1+y2)/2)
                xc=int(x2-x2)
                if yc<2000:
                    yt1=0
                    yt2=yc
                else:
                    yt1=yc
                    yt2=og.image.array.shape[1]
                trail_arr=np.zeros_like(check.image.array[yt1:yt2,x1:x2])
                trail_arr[:,:]=persist
                check.image.array[y1:y2,x1:x2]=check.image.array[y1:y2,x1:x2]+float_arr
                check.image.array[yt1:yt2,x1:x2]=check.image.array[yt1:yt2,x1:x2]+trail_arr
                dark[yt1:yt2,x1:x2]=trail_arr
                dark[y1:y2,x1:x2]=float_arr
            times[ind]+=30
        times.append(0)
        fptset=saturated_footprints(check)
        fptset_all.append(fptset)
        butler_output.put(check, 'raw_modified', datasetRefs_2[i].dataId)

This is then run through the pipeline via this yaml file:

Pipeline command is: